In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sns 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn import metrics
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import math
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, ParameterGrid

from utils import eval_model
from itertools import chain

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (4, 3)
mpl.rcParams['axes.grid'] = False

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [49]:
df_aluminium = pd.read_csv('dataset/' + '2005-2022-aluminium.csv')
df_copper = pd.read_csv('dataset/' + '2005-2022-copper.csv')
df_iron = pd.read_csv('dataset/' + '2010-2022-iron.csv')
df_palladium = pd.read_csv('dataset/' + '2005-2022-palladium.csv')
df_zinc = pd.read_csv('dataset/' + '2005-2022-zinc.csv')
df_lead = pd.read_csv('dataset/' + '2005-2022-lead.csv')
df_nickel = pd.read_csv('dataset/' + '2005-2022-nickel.csv')
df_platinum = pd.read_csv('dataset/' + '2005-2022-platinum_nymex.csv')
df_rhodium = pd.read_csv('dataset/' + '2005-2022-rhodium.csv')
df_tin = pd.read_csv('dataset/' + '2005-2022-tin.csv')
df_palladium_lbma = pd.read_csv('dataset/' + '2005-2022-palladium_lbma.csv')
df_platinum_lbma = pd.read_csv('dataset/' + '2005-2022-platinum_lbma.csv')
df_cobalt = pd.read_csv('dataset/' + '2010-2022-cobalt.csv')
df_rebar = pd.read_csv('dataset/' + '2009-2022-rebar_shfe.csv')
df_silver = pd.read_csv('dataset/' + '2005-2022-silver.csv')

dataset_all = [df_aluminium, df_copper, df_iron, df_palladium, df_zinc, df_lead, df_nickel, df_platinum, df_tin, df_palladium_lbma, df_platinum_lbma,df_rebar, df_silver]
name_all = ['aluminium', 'copper', 'iron' ,'palladium', 'zinc', 'lead', 'nickel', 'platinum', 'tin', 'palladium_lbma', 'platinum_lbma','rebar', 'silver']


common_cols = set.intersection(*map(set,dataset_all))

for df in dataset_all:
    df.drop(df.columns.difference(common_cols), axis=1, inplace=True)
    if df.columns[0] != 'date' or df.columns[-1] !='y':
        print('something wrong')
    
print('data cleaned')

data cleaned


## Linear Regression

### backtest and store evaluation metrics

In [61]:
def make_dataset_ml(df):
    # lookback = 66
    test_start_index = df[df.date >= '2019-05-01'].index[0]
    val_start_index = df[df.date >= '2016-05-01'].index[0]+22

    train_df = df[df.date < '2016-05-01']
    val_df = df.iloc[val_start_index:df[df.date >= '2019-05-01'].index[0]-22]
    test_df = df.iloc[test_start_index:]

    training_set = train_df.loc[:, (train_df.columns != 'date')].values
    val_set = val_df.loc[:, (val_df.columns != 'date')].values
    test_set = test_df.loc[:, (test_df.columns != 'date')].values

    y_date = df[df.date >= '2019-05-01']['date'].values
    
    X_train = train_df.loc[:, (train_df.columns != 'date') & (train_df.columns != 'y')].values
    y_train = train_df.loc[:, (train_df.columns != 'date') & (train_df.columns == 'y')].values

    X_val = val_df.loc[:, (val_df.columns != 'date') & (val_df.columns != 'y')].values
    y_val = val_df.loc[:, (val_df.columns != 'date') & (val_df.columns == 'y')].values

    X_test  = test_df.loc[:, (test_df.columns != 'date') & (test_df.columns != 'y')].values
    y_test = test_df.loc[:, (test_df.columns != 'date') & (test_df.columns == 'y')].values
    test_date = test_df['date'].values

    print('X_train:{}, y_train:{}, X_val:{}, y_val:{}, X_test:{}, y_test{}\n'.format(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape))

    return X_train, y_train, X_val, y_val, X_test, y_test

In [62]:
def LR_fixed(X_train, y_train, X_val, y_val, X_test, y_test):
    best_mse =1e9
    for n in range(1,10):
        scaler = StandardScaler()
        pca = PCA(n_components=n)

        X_train_transformed = scaler.fit_transform(X_train)
        X_train_transformed_pca = pca.fit_transform(X_train_transformed)
        # Linear Regression model
        regressor = LinearRegression()  
        regressor.fit(X_train_transformed_pca, y_train)

        X_val_transformed = scaler.transform(X_val)
        X_val_transformed_pca = pca.transform(X_val_transformed)
        
        # make prediction
        y_pred = regressor.predict(X_val_transformed_pca)
        _, mse, _, pcc = eval_model(y_pred.flatten(), y_val)

        if mse < best_mse:
                best_mse = mse
                best_n = n
    print('best n for pcr: ', best_n)
    
    
    scaler = StandardScaler()
    pca = PCA(n_components=best_n)
    
    X_train_transformed = scaler.fit_transform(X_train)
    X_train_transformed_pca = pca.fit_transform(X_train_transformed)
    
    X_test_transformed = scaler.transform(X_test)
    X_test_transformed_pca = pca.transform(X_test_transformed)
    
    
    regressor = LinearRegression()  
    regressor.fit(X_train_transformed_pca, y_train)
    # make prediction
    y_pred = regressor.predict(X_test_transformed_pca)
    print('LR: ', eval_model(y_pred.flatten(),y_test.flatten()))
    
    return eval_model(y_pred.flatten(),y_test.flatten())

In [63]:
def PLS_fixed(X_train, y_train, X_val, y_val, X_test, y_test):
    best_mse = 1e9
    for n in range(1,6):
        regressor = PLSRegression(n_components=n)
        regressor.fit(X_train, y_train)
        
        # make prediction
        y_pred = regressor.predict(X_val)
        _, mse, _, pcc = eval_model(y_pred.flatten(), y_val)

        if mse < best_mse:
                best_mse = mse
                best_n = n
    print('best n for pls: ', best_n)
    regressor = PLSRegression(n_components=best_n)
    regressor.fit(X_train, y_train)
    # make prediction
    y_pred = regressor.predict(X_test)

    print('PLS: ', eval_model(y_pred.flatten(),y_test.flatten()))
    
    return eval_model(y_pred.flatten(),y_test.flatten())

In [64]:
def XGB_fixed(X_train, y_train, X_val, y_val, X_test, y_test):
        grid = { 'max_depth': [3,6,10],
                'learning_rate': [0.01, 0.05, 0.1],
                'n_estimators': [100, 500, 1000],
                'colsample_bytree': [0.3, 0.7]}
        
        best_mse = 1e9
        for g in ParameterGrid(grid):

                # scaler = StandardScaler()
                # pca = PCA(n_components=5)

                # X_train_transformed = scaler.fit_transform(X_train)
                # X_train_transformed_pca = pca.fit_transform(X_train_transformed)

                regressor = XGBRegressor()
                regressor.set_params(**g)  
                regressor.fit(X_train, y_train)

                # X_val_transformed = scaler.transform(X_val)
                # X_val_transformed_pca = pca.transform(X_val_transformed)

                # make prediction
                y_pred = regressor.predict(X_val)
                _, mse, _, pcc = eval_model(y_pred.flatten(), y_val)

                if mse < best_mse:
                        best_mse = mse
                        best_grid = g
        regressor.set_params(**best_grid)  
        print(best_grid)
        regressor.fit(X_train, y_train)

        # X_test_transformed = scaler.transform(X_test)
        # X_test_transformed_pca = pca.transform(X_test_transformed)
                
        # make prediction
        y_pred = regressor.predict(X_test)

        print('XGB: ', eval_model(y_pred.flatten(),y_test.flatten()))
        
        return eval_model(y_pred.flatten(),y_test.flatten())

In [65]:
def SVR_fixed(X_train, y_train, X_val, y_val, X_test, y_test):
        grid = {'C': [math.pow(2,-5), math.pow(2,-3), math.pow(2,-1), math.pow(2,1), math.pow(2,3), math.pow(2,5)], 
                'gamma': [math.pow(2,-5), math.pow(2,-3), math.pow(2,-1), math.pow(2,1), math.pow(2,3)],
                'epsilon': [math.pow(2,-4), math.pow(2,-3), math.pow(2,-2), math.pow(2,-1)],
                'kernel': ['rbf','linear']}
        
        best_mse = 1e9
        for g in ParameterGrid(grid):

                # scaler = StandardScaler()
                # pca = PCA(n_components=5)

                # X_train_transformed = scaler.fit_transform(X_train)
                # X_train_transformed_pca = pca.fit_transform(X_train_transformed)

                regressor = SVR()
                regressor.set_params(**g)  
                regressor.fit(X_train, y_train)

                # X_val_transformed = scaler.transform(X_val)
                # X_val_transformed_pca = pca.transform(X_val_transformed)

                # make prediction
                y_pred = regressor.predict(X_val)
                _, mse, _, pcc = eval_model(y_pred, y_val)

                if mse < best_mse:
                        best_mse = mse
                        best_grid = g
        regressor.set_params(**best_grid)  
        print(best_grid)
        regressor.fit(X_train, y_train)

        # X_test_transformed = scaler.transform(X_test)
        # X_test_transformed_pca = pca.transform(X_test_transformed)
                

        # make prediction
        y_pred = regressor.predict(X_test)

        print('SVR',eval_model(y_pred.flatten(),y_test.flatten()))
        return eval_model(y_pred.flatten(),y_test.flatten())

In [77]:
dataset_all = [df_aluminium, df_copper, df_iron, df_palladium, df_zinc, df_lead, df_nickel, df_platinum, df_rhodium, df_tin, df_rebar, df_palladium_lbma, df_platinum_lbma, df_silver]
name_all = ['aluminium', 'copper', 'iron', 'palladium', 'zinc', 'lead', 'nickel', 'platinum', 'rhodium', 'tin', 'rebar', 'palladium_lbma', 'platinum_lbma', 'silver']

res_lr = []
res_pls = []
res_svr = []
res_xgb = []

for df, name in zip(dataset_all, name_all):
    print(name)
    print('------------------------------------------')
    X_train, y_train, X_val, y_val, X_test, y_test = make_dataset_ml(df)
    acc_lr, mse_lr,_, pcc_lr = LR_fixed(X_train, y_train, X_val, y_val, X_test, y_test)
    acc_pls, mse_pls,_, pcc_pls = PLS_fixed(X_train, y_train, X_val, y_val, X_test, y_test)
    acc_svr, mse_svr,_, pcc_svr = SVR_fixed(X_train, y_train, X_val, y_val, X_test, y_test)
    acc_xgb, mse_xgb,_, pcc_xgb = XGB_fixed(X_train, y_train, X_val, y_val,X_test, y_test)
    
    res_lr.append((acc_lr,mse_lr,pcc_lr))
    res_pls.append((acc_pls,mse_pls,pcc_pls))
    res_svr.append((acc_svr,mse_svr,pcc_svr))
    res_xgb.append((acc_xgb,mse_xgb,pcc_xgb))
    # y_pred_ensemble_ml = np.mean(np.array([y_pred_lr, y_pred_pls, y_pred_svr, y_pred_xgb]), axis=0)
    # print('ensemble ML: ', eval_model(y_pred_ensemble_ml.flatten(), y_test.flatten()))
    
    # df_t = pd.read_pickle('/home/yicheng.wang/UCL-Dissertation/models/LSTM-mse-{}-128units/{}-ypred.pkl'.format(name,name))
    # y_pred_ensemble_nn = np.mean(df_t['commodity_0'].values, axis=0)
    # print('ensemble NN: ', eval_model(y_pred_ensemble_nn.flatten(), y_test.flatten()))
    
    # y_pred_ensemble_again = np.mean(np.array([y_pred_lr, y_pred_pls, y_pred_svr, y_pred_xgb, y_pred_ensemble_nn]), axis=0)
    # print('ensemble NN: ', eval_model(y_pred_ensemble_again.flatten(), y_test.flatten()))
    print('\n\n')

aluminium
------------------------------------------
X_train:(2870, 11), y_train:(2870, 1), X_val:(738, 11), y_val:(738, 1), X_test:(740, 11), y_test(740, 1)

best n for pcr:  2
LR:  (0.65, 0.0030580246774202817, -7.285345150718067, 0.26462038271826765)
best n for pls:  5
PLS:  (0.677027027027027, 0.003062993027441772, -5.517063741563463, 0.30301354394590074)
{'C': 32.0, 'epsilon': 0.125, 'gamma': 0.03125, 'kernel': 'rbf'}
SVR (0.41621621621621624, 0.04239338308396595, -0.7243421987453271, -0.22074555079386377)
{'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
XGB:  (0.7135135135135136, 0.0031439593855431647, -2.0746193928395287, 0.3543971687318296)



copper
------------------------------------------
X_train:(2870, 11), y_train:(2870, 1), X_val:(738, 11), y_val:(738, 1), X_test:(740, 11), y_test(740, 1)

best n for pcr:  6
LR:  (0.5324324324324324, 0.0037735380224055244, -5.533109975029702, 0.1688427253974983)
best n for pls:  1
PLS:  (0.49594594594